## Introduction to Conversational AI with MLflow and DialoGPT


In [7]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category = UserWarning)

env: TOKENIZERS_PARALLELISM=false


## Setting Up th Conversational Pipeline

In [8]:
import mlflow.models
import mlflow.models
import transformers
import mlflow 

# Define our pipeline 
conversational_pipeline = transformers.pipeline(model = "microsoft/DialoGPT-medium")

# Infer the signature
signature = mlflow.models.infer_signature(
    "Hi there, Chatbot!",
    mlflow.transformers.generate_signature_output(conversational_pipeline, "Hi there, Another example")
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## Creating an Experiment 

In [9]:
mlflow.set_experiment("Conversational")

2024/11/18 15:28:35 INFO mlflow.tracking.fluent: Experiment with name 'Conversational' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///e:/MLFlow/mlruns/186907240044108028', creation_time=1731923915866, experiment_id='186907240044108028', last_update_time=1731923915866, lifecycle_stage='active', name='Conversational', tags={}>

In [10]:
# Set our tracking uri for logging
mlflow.set_tracking_uri(uri = "http://127.0.0.1:8080")

## Logging the Model with MLflow

In [19]:
with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=conversational_pipeline,
        artifact_path="chatbot",
        task="conversational",
        signature=signature
    )

2024/11/18 15:43:16 WARNING mlflow.utils.environment: On Windows, timeout is not supported for model requirement inference. Therefore, the operation is not bound by a timeout and may hang indefinitely. If it hangs, please consider specifying the signature manually.
2024/11/18 15:43:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run intrigued-conch-438 at: http://127.0.0.1:8080/#/experiments/186907240044108028/runs/055adab0f56e4fbd8335ce8c79b717ee.
2024/11/18 15:43:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/186907240044108028.


## Loading and Interacting with the Chatbot Model 

In [ ]:
from transformers import pipeline
from transformers import Conversation

# Create a conversational pipeline
chat_pipeline = pipeline("conversational", model = "microsoft/DialoGPT-medium")

conversation = Conversation("What is the best way to get to Antarctica?")
response = chat_pipeline([conversation])
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: f66ce8d9-1be9-492f-b6ed-50392c2f46c4
user: What is the best way to get to Antarctica?
assistant: I think you can get there by boat.



In [51]:
# Load the model as a generic python function in order to leverage the integrated Conversational Context
# Note that loading a conversational model with the native flavor (i.e., `mlflow.transformers.load_model()`) will not include anything apart from the
# pipeline itself; if choosing to load in this way, you will need to manage your own Conversational Context instance to maintain state on the
# conversation history.
chatbot = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

# Validate that the model is capable of responding to a question
first = chatbot.predict("What is the best way to get to Antarctica?")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [52]:
print(f"Response: {first}")

Response: ['What is the best way to get to Antarctica?']


In [53]:
# Verify that the PyFunc implementation has maintained state on the conversation history by asking a vague follow-up question that requires context
# in order to answer properly
second = chatbot.predict("What sort of boat should I use?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [54]:
print(f"Response: {second}")

Response: ['What sort of boat should I use?']
